<b>Código principal para cargar datos.</b>
<p> Configurar fechas de carga, seriales de boyas y rutas para archivos en \Configs\configuracion_general.py </p>


In [ ]:
####################### N O  T O C A R ############################################
%load_ext autoreload
%autoreload 2
import sys
import os

# Agrega la ruta raíz del proyecto si no está
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

# import Configs.configuracion_general as config
import Funciones.Carga.cargar_datos_csv as carga 
from Funciones.Utils.utilidades import guardar_diccionario_como_pickle
from Funciones.Correctores.corrector_utils import eliminar_datos_espurios
from Funciones.Utils.utils_get_config_vars import * 
##################################################################################

In [ ]:
rutas_de_sondas, seriales_encontrados = carga.buscar_nombre_de_archivo_de_sonda()


In [ ]:
diccionario_de_datos_de_sondas = carga.cargar_datos_de_sonda(rutas_de_sondas, seriales_encontrados)

In [ ]:
# diccionario_de_datos_de_sondas["4878196"].head()

In [ ]:
df_excel_de_despliegue = carga.leer_excel_de_despliegue_de_sondas(seriales_encontrados)

In [ ]:
# df_excel_de_despliegue

In [ ]:
diccionario_de_sondas_en_fechas = carga.seleccionar_rango_de_fechas(diccionario_de_datos_de_sondas, df_excel_de_despliegue, seriales_encontrados)
datos_de_sondas_sin_duplicados = carga.buscar_y_eliminar_duplicados(diccionario_de_sondas_en_fechas, seriales_encontrados)
datos_ordenados = carga.ordernar_datos_por_fecha(datos_de_sondas_sin_duplicados, seriales_encontrados)

In [ ]:
datos_con_tspan_redondeado = carga.crear_tspan_redondeado(datos_de_sondas_sin_duplicados, seriales_encontrados) # diccionario con tspan redondeado
carga.existen_fechas_redondeadas_duplicadas(datos_con_tspan_redondeado, seriales_encontrados) # Verifica si existen fechas redondeadas duplicadas, si existen lanza un error

In [ ]:
# datos_con_tspan_redondeado[seriales_de_sondas[0]].head(10) # Muestra de los datos de la primera sonda cualquiera

In [ ]:
# A continuación se crea el dataframe vacío con todas las fechas que pueden existir y se combina con el dataframe de cada sonda
columnas = carga.get_cols_names_sin_tspan(diccionario = datos_con_tspan_redondeado)
diccionario_con_dfs_vacios = carga.crear_diccionario_con_dataframes_vacios(diccionario= datos_con_tspan_redondeado, keys=columnas)
datos_finales = carga.merge_df_vacio_con_datos(datos_con_tspan_redondeado, diccionario_con_dfs_vacios)

In [ ]:
# datos_finales["4878205"].head(10) # Ejemplo de los datos de salida

In [ ]:
# Eliminar datos espurios (solo se revisa si hay valores de rapidez superiores a 2 m/s y se elimina toda la fila)
datos_finales = eliminar_datos_espurios(datos_finales)

In [ ]:
# Agregar componentes de la velocidad al diccionario con los dataframe de cada sonda
datos_finales = carga.agregar_componentes_de_la_velocidad(datos_finales)

In [ ]:
# datos_finales["4878196"].head()

In [ ]:
# Guardar datos de salida en archivo .pkl
guardar_diccionario_como_pickle(datos_finales, ruta = get_carpeta_guardado_datos_procesados(), nombre_archivo=get_nombre_archivo_datos_procesados())